In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

In [2]:
data = pd.read_excel("/content/sample_data/Load_Data_in_kW.xlsx",header=None)
data

,0
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980
...,...
2179,4626.51840
2180,4567.90752
2181,4723.63164
2182,5150.27520


In [3]:
df = data
q = df[0].quantile(0.99)
df[df[0] < q]
q_low = df[0].quantile(0.01)
q_hi  = df[0].quantile(0.99)

df_filter= df[(df[0] < q_hi) & (df[0] > q_low)]
df_filter

,0
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980
...,...
2179,4626.51840
2180,4567.90752
2181,4723.63164
2182,5150.27520


In [4]:
print(len(data))
print(len(df_filter))
if(len(data) == len(df_filter)):
    print('No outliners')

2184
2140


In [5]:
#normalize
data = df_filter
data_n = data.copy()
data_n = (data_n - data_n.min())/(data_n.max() - data_n.min())
data_n

,0
0,0.372950
1,0.254941
2,0.235273
3,0.273574
4,0.299162
...,...
2179,0.180927
2180,0.168764
2181,0.201080
2182,0.289619


In [6]:
train_data,test_data = train_test_split(data_n,test_size=0.1)

In [7]:
print(len(train_data))
train_data

1926


,0
1496,0.281425
1362,0.231650
1343,0.818685
1753,0.314981
1761,0.580112
...,...
77,0.565878
1307,0.825154
565,0.532494
383,0.391389


In [9]:
xar = []
yar = []
xar1 = []
for i in range(24,len(train_data.iloc[:,0])):
    xar.append(train_data.iloc[i-24,0])
    yar.append(train_data.iloc[i,0])
    xar1.append(train_data.iloc[i-1,0])

In [10]:
from sympy import *
w1= Symbol('m1')
w2= Symbol('m2')
c= Symbol('c')
function = '0.5*(y-m1*x1-m2*x2-c)**2'
partialderiv= Derivative(function, w1)
dfm1 = partialderiv.doit()
partialderiv= Derivative(function, w2)
dfm2 = partialderiv.doit()
partialderiv= Derivative(function, c)
dfc = partialderiv.doit()
err = []
print(f'first order derivatives of given function are de/dm1 = {dfm1},de/dm2 = {dfm2} ,de/dc = {dfc}')

#step 1 
m1 = 1.0 
m2 = 1.0
c1 = -1.0  
itr = 20 
EM1 = 0
EM2 = 0
EC = 0
ep = 10**(-8)
gama = 0.9
learning_rate = 0.1  
for i in range(0,itr):
    er = 0
    for j in range(0,len(xar)):
        er +=  ((yar[j] - (m1 * xar[j])-(m2 * xar1[j]) - c1))**2
        #step 2 
        w1= Symbol('m1')
        w2= Symbol('m2')
        c = symbols('c')
        x1 = symbols('x1')
        x2 = symbols('x2')
        y = symbols('y')
        dfm1v = dfm1.subs(w1, m1)
        dfm1v = dfm1v.subs(w2,m2)
        dfm1v = dfm1v.subs(c, c1) 
        dfm1v = dfm1v.subs(x1, xar[j])
        dfm1v = dfm1v.subs(x2, xar1[j])
        dfm1v = dfm1v.subs(y, yar[j])         
        dfm2v = dfm2.subs(w2, m2)
        dfm2v = dfm2v.subs(w1,m1)
        dfm2v = dfm2v.subs(c, c1) 
        dfm2v = dfm2v.subs(x1, xar[j])
        dfm2v = dfm2v.subs(x2, xar1[j])
        dfm2v = dfm2v.subs(y, yar[j]) 
        dfm2v = round(dfm2v,2)
        dfcv = dfc.subs(c, c1)   
        dfcv = dfcv.subs(w2, m2)
        dfcv = dfcv.subs(w1,m1)
        dfcv = dfcv.subs(x1, xar[j])
        dfcv = dfcv.subs(x2, xar1[j])
        dfcv = dfcv.subs(y, yar[j]) 
        dfcv = round(dfcv,2)
        
        EM1 = gama*EM1 + (1-gama)*(dfm1v**2)        
        EM2 = gama*EM2 + (1-gama)*(dfm2v**2)
        EC = gama*EC + (1-gama)*(dfcv**2)
        #step3 
        dm1 = (-1.0)*(learning_rate/(EM1+ep)**(0.5))*dfm1v
        dm2 = (-1.0)*(learning_rate/(EM2+ep)**(0.5))*dfm2v
        dc = (-1.0)*(learning_rate/(EC+ep)**(0.5))*dfcv
        #step4 
        m1 = m1 + dm1
        m1 = round(m1, 2)
        m2 = m2 + dm2
        m2 = round(m2, 2)
        c1 = c1 + dc
        c1 = round(c1, 2)
        
        er = (1/(2*len(xar)))*(er)
        err.append(er)
print(f'minimum value obtained at m1 = {m1} ,m2 = {m2} ,c ={c1} for given function')

first order derivatives of given function are de/dm1 = -1.0*x1*(-c - m1*x1 - m2*x2 + y),de/dm2 = -1.0*x2*(-c - m1*x1 - m2*x2 + y) ,de/dc = 1.0*c + 1.0*m1*x1 + 1.0*m2*x2 - 1.0*y
minimum value obtained at m1 = 0.05 ,m2 = -0.70 ,c =0.66 for given function


In [11]:
#RMSE - Training data
train_data_pred = [i for i in xar]
for k in range(len(xar)):
    train_data_pred[k]=(m1*xar[k]*xar1[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(yar,train_data_pred)
mse

0.08493680742219464

In [12]:
#Test Data
xar = []
yar = []
for i in range(len(test_data.iloc[:,0])-24):
    xar.append(test_data.iloc[i,0])
    yar.append(test_data.iloc[i+24,0])
test_data_pred = [i for i in xar]
for k in range(len(xar)):
    test_data_pred[k]=(m1*xar[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(yar,test_data_pred)
mse

0.07719318371419447

In [13]:
x_max = y_max= 8841.669480
x_min = y_min = 3377.919600
x =x1 = float(input("load of previous hour :"))
x = (x - x_min)/(x_max - x_min)
y = m1*x + c1
y = y*(y_max - y_min) + y_min
print(f"load at hour based on its previous hour is : {y}")

load of previous hour :5050
load at hour based on its previous hour is : 7068.12567351563
